In [1]:
using Pkg 
Pkg.activate(".")

  Activating new project at `c:\Users\USAID\Documents\2023-2024\phys 218\NonLinearDynamics\Julia Practice`


# Types

In Juia, the type system follows a certain hierarchy that can be thaught of as a tree. The root of the tree is the `Any` type, which is the supertype of all types. For exemple the `Any` type has two subtypes: `Number` and `String`. The `Number` type has two subtypes: `Integer` and `Float`. The `String` type has no subtypes. The types that are at the bottom of the tree are on leaf node.

In [4]:
subtypes(Float64)

Type[]

In [5]:
supertype(Float64)

AbstractFloat

In [6]:
supertype(AbstractFloat)

Real

In [7]:
supertype(Real)

Number

In [8]:
using AbstractTrees;
AbstractTrees.children(x::Type) = subtypes(x)

In [9]:
print_tree(Number)

Number


├─ MultiplicativeInverse
│  

├─ SignedMultiplicativeInverse
│  └─ UnsignedMultiplicativeInverse
├─ Complex
├─ Real
│  ├─ AbstractFloat
│  │  

├─ BigFloat
│  │  ├─ Float16
│  │  ├─ Float32
│  │  └─ Float64
│  ├─ AbstractIrrational
│  │  └─ Irrational
│  ├─ Integer
│  │  ├─ Bool
│  │  ├─ Signed
│  │  │  ├─ BigInt
│  │  │  ├─ Int128
│  │  │  ├─ Int16
│  │  │  ├─ Int32
│  │  │  ├─ Int64
│  │  │  └─ Int8
│  │  └─ Unsigned
│  │     

├─ UInt128
│  │     ├─ UInt16
│  │     ├─ UInt32
│  │     ├─ UInt64
│  │     └─ UInt8
│  └─ Rational
├─ AbstractQuantity
│  └─ Quantity
└─ LogScaled
   ├─ Gain{L} where L<:LogInfo
   └─ Level{L} where L<:LogInfo


I will now introduce a new operator: the `<:` operator which is used to check if a type is a subtype of another type. For exemple, `Number <: Any` is true, but `Any <: Number` is false. The `<:` operator is also used to check if a value is of a certain type. For exemple, `1 <: Number` is true, but `"1" <: Number` is false.

In [10]:
Float64 <: Number

true

In [11]:
Float64 <: Int

false

## Abstract and concrete types

You can't create an instance of an abstract type. For exemple, you can't create an instance of the `Number` or `Real` types. You can only create instances of concrete types. For exemple, you can create an instance of the `Int64` type, which is a subtype of `Integer`.

# Custom types

It is very usefull to construct your own types. You can do this using the `struct` and `mutable` key words.

In [12]:
struct A
    x
    y
end
mutable struct B
    x
    y
end

In [13]:
a = A(1,2)

A(1, 2)

In [14]:
b = B(1,2)

B(1, 2)

You can access the feilds of a struct using the `.` operator.

In [15]:
a.x

1

Since B is mutable, you can change the value of its feilds.

In [16]:
b.x = 3
b

B(3, 2)

But for a it will not work

In [17]:
a.x = 3

ErrorException: setfield!: immutable struct of type A cannot be changed

This is very bad tho, since the compiler can't know what type the feilds of a are. So instead of using `Any` as the type of the feilds, we can specify the type of the feilds.

In [18]:
struct A_Better
    x::Int
    y::Int
    z::String
end

# Abstract types

In [19]:
abstract type C end

In [20]:
struct D <: C
    x::Number
    y::Number
    z::String
end

In [21]:
subtypes(C)

1-element Vector{Any}:
 D

This is still bad because D is type unstable. The compiler can't know what type the feilds of D are.
But what if we need flexibility?

# Generic/ Parametric types

In [1]:
struct Gen{T<: Number} 
    x::T
    y::T
    z::T
end

In [3]:
Gen{Float64}(1,2,3)

Gen{Int64}(1, 2, 3)

In [24]:
Gen{Int}(1.0,2,3)

Gen{Int64}(1, 2, 3)

What if the types could be different in one struct? 

In [25]:
struct Gen2{T,F} 
    x::T
    y::F
end

In [26]:
Gen2{String,Float64}("Hello",64)

Gen2{String, Float64}("Hello", 64.0)

In [27]:
Gen2("Hello",64)

Gen2{String, Int64}("Hello", 64)

Last example

In [4]:
struct Z{T <: Real, F <: Union{Nothing, String}}
    x::T
    z::Int64
    y::F
end

In [7]:
Z{Float64,String}(1.0,3,"hello")

Z{Float64, String}(1.0, 3, "hello")

In [9]:
Z(1.0,3,nothing)

Z{Float64, Nothing}(1.0, 3, nothing)

# Multiple dispatch or Overloading

In python you can't have two functions with the same name. But in Julia you can. This is called multiple dispatch. The function that is called depends on the types of the arguments. This is very usefull because you can have the same function name for different types. For exemple, you can have a `+` function for integers and a `+` function for floats.


In python the method is determined by what class the object is an instance of. In Julia, the method is determined by the types of the arguments.

In [31]:
set_size(x::Real, y::Real) = ... 
set_size(x::Real, y::Real, z::Real) = ...
set_size(x::String, y::Real) = ... 
set_size(x::Real, y::String) = ... 


Base.Meta.ParseError: ParseError:
# Error @ /Users/jason/Desktop/Spring 2024/Julia_Crash_Course/Course/Day2/types.ipynb:1:30
set_size(x::Real, y::Real) = ... 
#                            └─┘ ── invalid identifier

Meaning you have infinite number of methods for a function.

## How to make sure that you are calling the right method?

Julia will always try to find the method that has the most specific types. For exemple,

In [32]:
set_size(A::Vector) = ...
set_size(B::Vector{<:Real}) = ...
set_size(C::Vector{Union{String,Float64}}) = ...
set_size(C::Vector{Float64}) = ...

Base.Meta.ParseError: ParseError:
# Error @ /Users/jason/Desktop/Spring 2024/Julia_Crash_Course/Course/Day2/types.ipynb:1:23
set_size(A::Vector) = ...
#                     └─┘ ── invalid identifier

## Notice

Method do not belong to the type and so you can define methods after you created the type

In [33]:
abstract type Animals end

struct Dog <: Animals
    name::String
    age::Int
end
struct Cat <: Animals
    name::String
    age::Int
end


In [34]:
function encounter(x::Animals, y::Animals)
    verb = meets(x,y)
    println("$(x.name) meets $(y.name) and they $verb")
end
meets(x::Animals, y::Animals) = "pass by"

meets (generic function with 1 method)

In [35]:
Fido = Dog("Fido", 3)
Rex = Dog("Rex", 5)
Mittens = Cat("Mittens", 2);
Fluffy = Cat("Fluffy", 4);


In [36]:
encounter(Fido, Mittens)

Fido meets Mittens and they pass by


In [37]:
meets(x::Dog, y::Cat) = "fight"
meets(x::Cat, y::Dog) = "fight"
meets(x::Dog, y::Dog) = "play"
meets(x::Cat, y::Cat) = "play"

meets (generic function with 5 methods)

In [38]:
encounter(Fido, Mittens)
encounter(Fido, Rex)
encounter(Mittens, Fluffy)
encounter(Fluffy, Mittens)


Fido meets Mittens and they fight
Fido meets Rex and they play
Mittens meets Fluffy and they play
Fluffy meets Mittens and they play


# Easy to extend

In [39]:
struct Rabbit <: Animals
    name::String
    age::Int
end

In [40]:
meets(x::Dog, y::Rabbit) = "chase"
meets(x::Rabbit, y::Cat) = "hide"

meets (generic function with 7 methods)

In [41]:
fluffy = Rabbit("Fluffy", 1)

Rabbit("Fluffy", 1)

In [42]:
encounter(Rex, fluffy)
encounter(fluffy, Mittens)
encounter(fluffy, Rex)

Rex meets Fluffy and they chase
Fluffy meets Mittens and they hide
Fluffy meets Rex and they pass by


Making sure of what function is called when you have a large package that you don't know

In [43]:
methods(meets)

# 7 methods for generic function "meets" from Main:
 [1] meets(x::Rabbit, y::Cat)
     @ ~/Desktop/Spring 2024/Julia_Crash_Course/Course/Day2/types.ipynb:2
 [2] meets(x::Dog, y::Rabbit)
     @ ~/Desktop/Spring 2024/Julia_Crash_Course/Course/Day2/types.ipynb:1
 [3] meets(x::Cat, y::Cat)
     @ ~/Desktop/Spring 2024/Julia_Crash_Course/Course/Day2/types.ipynb:4
 [4] meets(x::Dog, y::Dog)
     @ ~/Desktop/Spring 2024/Julia_Crash_Course/Course/Day2/types.ipynb:3
 [5] meets(x::Cat, y::Dog)
     @ ~/Desktop/Spring 2024/Julia_Crash_Course/Course/Day2/types.ipynb:2
 [6] meets(x::Dog, y::Cat)
     @ ~/Desktop/Spring 2024/Julia_Crash_Course/Course/Day2/types.ipynb:1
 [7] meets(x::Animals, y::Animals)
     @ ~/Desktop/Spring 2024/Julia_Crash_Course/Course/Day2/types.ipynb:5

In [44]:
@which meets(Rex, fluffy)

meets(x::Dog, y::Rabbit)
     @ Main ~/Desktop/Spring 2024/Julia_Crash_Course/Course/Day2/types.ipynb:1

# Everything is a method in Julia


In [45]:
+

+ (generic function with 203 methods)

In [46]:
methods(+)

# 203 methods for generic function "+" from Base:
   [1] +(level::Base.CoreLogging.LogLevel, inc::Integer)
     @ Base.CoreLogging logging.jl:131
   [2] +(x::Bool, z::Complex{Bool})
     @ complex.jl:305
   [3] +(x::Bool, y::Bool)
     @ bool.jl:166
   [4] +(x::Bool)
     @ bool.jl:163
   [5] +(x::Bool, z::Complex)
     @ complex.jl:312
   [6] +(x::Real, z::Complex{Bool})
     @ complex.jl:319
   [7] +(x::Bool, y::T) where T<:AbstractFloat
     @ bool.jl:173
   [8] +(x::Dates.DateTime, y::Dates.Quarter)
     @ Dates /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:77
   [9] +(dt::Dates.DateTime, z::Dates.Month)
     @ Dates /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:49
  [10] +(dt::Dates.DateTime, y::Dates.Year)
     @ Dates /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:25
  [11] +(x::Dates.DateTime, y::Dates.Period)
     @ Dates /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:83
  [12] +(::Missing, ::Missing)
     @ missing.jl:122
  [13] +(::Missing)
     @ missing.jl:101
  [14] +(x::Missing, y::Dates.AbstractTime)
     @ Dates /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:91
  [15] +(::Missing, ::Number)
     @ missing.jl:123
  [16] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod)
     @ Dates /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Dates/src/periods.jl:334
  [17] +(x::Dates.CompoundPeriod, y::Dates.TimeType)
     @ Dates /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Dates/src/periods.jl:362
  [18] +(x::Dates.CompoundPeriod, y::Dates.Period)
     @ Dates /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Dates/src/periods.jl:332
  [19] +(a::Pkg.Resolve.VersionWeight, b::Pkg.Resolve.VersionWeight)
     @ Pkg.Resolve /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Pkg/src/Resolve/versionweights.jl:22
  [20] +(x::Rational{BigInt}, y::Rational{BigInt})
     @ Base.GMP.MPQ gmp.jl:1061
  [21] +(c::BigInt, x::BigFloat)
     @ Base.MPFR mpfr.jl:468
  [22] +(x::BigInt, y::BigInt)
     @ Base.GMP gmp.jl:501
  [23] +(a::BigInt, b::BigInt, c::BigInt)
     @ Base.GMP gmp.jl:541
  [24] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt)
     @ Base.GMP gmp.jl:542
  [25] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt)
     @ Base.GMP gmp.jl:543
  [26] +(x::BigInt, y::BigInt, rest::BigInt...)
     @ Base.GMP gmp.jl:683
  [27] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8})
     @ Base.GMP gmp.jl:549
  [28] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8})
     @ Base.GMP gmp.jl:555
  [29] +(t::Dates.Time, dt::Dates.Date)
     @ Dates /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:22
  [30] +(x::Dates.Time, y::Dates.TimePeriod)
     @ Dates /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:85
  [31] +(a::Pkg.Resolve.FieldValue, b::Pkg.Resolve.FieldValue)
     @ Pkg.Resolve /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Pkg/src/Resolve/fieldvalues.jl:43
  [32] +(x::Dates.Date, y::Dates.Day)
     @ Dates /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:81
  [33] +(x::Dates.Date, y::Dates.Week)
     @ Dates /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:79
  [34] +(x::Dates.Date, y::Dates.Quarter)
     @ Dates /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:75
  [35] +(dt::Dates.Date, z::Dates.Month)
     @ Dates /Applications/Julia-1.10.0.app/Contents/Resources/julia/share/julia/stdlib/v1.10/Date

So something funny you can do is actually implement your own version of the `+` operator. For exemple, you can define a `+` operator that makes two animals encounter.

In [47]:
Base.:+(x::Animals, y::Animals) = encounter(x,y)

In [48]:
Fido + Mittens

Fido meets Mittens and they fight


# Dispatch with parametric types

In [49]:
struct Alpha{T}
    x::T
end

In [50]:
foo(x::Alpha) = "General form"
foo(x::Alpha{<:Real}) = "Specialized for Reals"
foo(x::Alpha{Int}) = "Specialized for Ints"
foo(x::Alpha{Int64}) = "Specialized for Ints64"

foo (generic function with 3 methods)

In [51]:
a1=Alpha("hello")
a2=Alpha(1)
a3=Alpha(1.0)

Alpha{Float64}(1.0)

In [52]:
println(foo(a1))
println(foo(a2))
println(foo(a3))

General form
Specialized for Ints64
Specialized for Reals


# Generic Algorithms

Imagine the following simple function that just computes the dot product os two vectors of the same size.

In [20]:
using LinearAlgebra

In [21]:
function mydot(x::AbstractVector, y::AbstractVector)
    x ⋅ y
end

mydot (generic function with 1 method)

In [22]:
x = ones(10)
y = ones(10)
mydot(x,y)

10.0

### One hot vector 

The one hot vector is a vector that has only one non zero element. For exemple, the one hot vector of size 5 with the third element being 1 is `[0, 0, 1, 0, 0]`. The one hot vector of size 5 with the first element being 1 is `[1, 0, 0, 0, 0]`.

We can implement the one hot vector using our own type.

In [23]:
struct OneHotVector <: AbstractVector{Bool}
    ind :: Int
    len :: Int
end

In [24]:
Base.size(v::OneHotVector) = (v.len,)
Base.getindex(v::OneHotVector, i::Int) = Int(i == v.ind)
Base.length(v::OneHotVector) = v.len

In [25]:
v = OneHotVector(3, 5)
v[3], v[4]

(1, 0)

In [26]:
v1 = OneHotVector(100, 1_000_000);
v2 = 1:1_000_000;

In [27]:
using BenchmarkTools

In [28]:
@btime mydot(v1,v2)

  178.027 μs (0 allocations: 0 bytes)


100

In [31]:
LinearAlgebra.dot(v1::OneHotVector,v2::AbstractArray) = v2[v1.ind] 

In [32]:
@btime mydot(v1,v2)

  29.059 ns (0 allocations: 0 bytes)


100